In [1]:
import tensorflow as tf
from tensorflow import keras
import IPython
import kerastuner as kt
from tqdm.keras import TqdmCallback

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Using TensorFlow backend.


In [2]:
# Es wird wieder der Fashion MNIST Datensatz verwendet, wie zuvor schon bei img Klassifikation

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [3]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step =32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4] )
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    
    return model

In [4]:
# Hyperband funktioniert wie ein tunierbracket. Es lässt modelle gegeneinander antretten und nur die gewinner kommen weiter
# So dass die beste hälfte in die nächste Runde geht usw.

tuner = kt.Hyperband(model_builder, objective = 'val_accuracy', max_epochs = 10, factor = 3, directory ='my_dir', project_name='intro_to_kt')

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

INFO:tensorflow:Reloading Oracle from existing project my_dir\intro_to_kt\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\intro_to_kt\tuner0.json


In [5]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""Suche fertig, erste dense Layer: {best_hps.get('units')} und optimale Lernrate ist: {best_hps.get('learning_rate')}""")

INFO:tensorflow:Oracle triggered exit
Suche fertig, erste dense Layer: 224 und optimale Lernrate ist: 0.001


In [6]:
# Trainieren des Modells mit den ermittelten Werten

model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), verbose=0, callbacks=[TqdmCallback(verbose=2)])